In [ ]:
import os
import yaml
import nbimporter
from datetime import datetime, date
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType,  DateType, TimestampType
from pyspark.sql.functions import to_date, to_timestamp
from lab_table_manager import TableManager
from lab_spark import create_spark_session
from lab_raw_yfinance import RawYFinance
import yfinance as yf
from lab_pg_database_manager import PGDatabaseManager

In [ ]:
def parallel_fetch_record(spark, fetch_function, fetch_function_params, record_columm_list):
    try:
        # Distribute (symbol, start_date) pairs across Spark workers
        param_rdd = spark.sparkContext.parallelize(fetch_function_params)    
        
        # Fetch data in parallel
        mapped_record_rdd = param_rdd.flatMap(fetch_function)

        # Convert RDD to DataFrame
        result_df = spark.createDataFrame(mapped_record_rdd, schema=record_columm_list)
        # result_df = spark.createDataFrame(mapped_record_rdd)

        # Show or save the results
        # result_df.show()
        return result_df
    except Exception as e:
        print(f"Error paralleling fetch: {e}")
        return spark.createDataFrame([]) 

In [3]:
connection_config_file="cfg_connections.yaml"
spark_app_name="raw_yfinance"
spark=create_spark_session(connection_config_file, spark_app_name)
equity_type ="stock"
symbol_date_pairs = [
    ('TSLA', '2024-11-20'),
    ('AAPL', '2024-11-20'),
    ('C', '2024-11-20'),
]
object_schema_config_file="cfg_table_schemas.yaml"
object_type="apis"
object_name="yfinance_stock"
raw_stock_yfiannce = RawYFinance(equity_type, symbol_date_pairs, object_schema_config_file, object_type, object_name)
col_list=['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
df = parallel_fetch_record(spark, raw_stock_yfiannce.fetch_yfinance_record, symbol_date_pairs, col_list)
df.show()



25/01/02 19:39:56 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/01/02 19:40:11 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/01/02 19:40:26 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/01/02 19:40:41 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/01/02 19:40:56 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/01/02 19:41:11 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure th

Error paralleling fetch: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 4 times, most recent failure: Lost task 0.3 in stage 0.0 (TID 3) (172.33.0.3 executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1227, in main
    func, profiler, deserializer, serializer = read_command(pickleSer, infile)
                                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 90, in read_command
    command = serializer._read_with_length(file)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 174, in _read_with_length
    return self.loads(obj)
           ^^^^^^^^^^^^^^^
  File "/opt/spark/python/lib/pyspark.zip/pyspark/ser

PySparkValueError: [CANNOT_INFER_EMPTY_SCHEMA] Can not infer schema from empty dataset.

In [ ]:
df.show()